In [ ]:
# 목표

#각 주택의 판매 가격을 예측하는 것은 당신의 일입니다. 테스트 세트의 각 Id에 대해 SalePrice 변수의 값을 예측해야 합니다.

# 미터법

# 제출은 예측 값의 로그와 관찰된 판매 가격의 로그 사이의 루트-Mean-Squared-오차(RMSE)로 평가됩니다. (로그를 기록한다는 것은 비싼 집과 싼 집을 예측하는 오류가 결과에 똑같이 영향을 미친다는 것을 의미한다.)

# 제출 파일 형식

# 파일에 헤더가 포함되어야 하며 다음과 같은 형식을 가져야 합니다.

# 아이디, 판매 가격
# 1461,169000.1
# 1462,187724.1233
# 1463,175221
# 등등.

아래에는 “House Prices” 문제에 대한 전처리·모델링 코드를 처음부터 스스로 떠올릴 수 있도록, 의사결정 과정을 단계별로 정리한 것입니다. 각 단계에서 어떤 고민이 필요한지, 어떤 키워드를 떠올려야 하는지 연습해 보세요.

⸻

	1.	문제 정의
	•	주어진 피처로 SalePrice를 예측하는 회귀(regression) 문제다.
	•	평가 지표(RMSE)에 익숙해져 두자.
	2.	필요 라이브러리 선정
	•	데이터 다루기: pandas, numpy
	•	전처리·모델링: scikit-learn(Pipeline, ColumnTransformer, SimpleImputer, StandardScaler, OneHotEncoder, RandomForestRegressor 등)
	3.	데이터 로드 및 원본 보존
	•	pd.read_csv()로 불러온 뒤, raw_train, raw_test에 저장.
	•	train = raw_train.copy(), test = raw_test.copy()로 복제본을 만들어 둔다.
	•	→ 언제든 초기 상태로 되돌릴 수 있도록.
	4.	데이터 탐색(EDA)
	•	train.shape, test.shape로 크기 확인
	•	train.columns, train.isnull().sum(), train.describe()로 결측치·분포 파악
	•	→ 어떤 컬럼에 결측이 많은지, 이상치(outlier)는 없는지 체크
	5.	피처 유형(feature types) 구분
	•	수치형(numeric): select_dtypes(include=[np.number])
	•	범주형(categorical): select_dtypes(include=['object'])
	•	→ int64·float64만 나오는 이유와, category 타입을 쓸지 고민
	6.	결측치 처리 전략 수립
	•	수치형: 대표값(median)으로 대체 → SimpleImputer(strategy='median')
	•	범주형: 새로운 범주 ‘missing’으로 채우기 → SimpleImputer(strategy='constant', fill_value='missing')
	•	→ NaN 때문에 모델 에러가 나지 않도록
	7.	스케일링·인코딩 전략 수립
	•	수치형: 서로 단위가 다른 피처를 표준화 → StandardScaler()
	•	범주형: 문자열 범주를 0/1 더미로 변환 → OneHotEncoder(handle_unknown='ignore')
	8.	개별 변환기(transformer) 정의

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',  StandardScaler()),
])
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore')),
])


	9.	ColumnTransformer 구성
	•	수치형 변환기 → numeric_features
	•	범주형 변환기 → categorical_features
	•	나머지 컬럼(Id 등)은 그대로 통과 → remainder='passthrough'
	10.	전체 Pipeline 구성
	•	첫 단계: preprocessor(ColumnTransformer)
	•	두 번째 단계: 모델(RandomForestRegressor 등)
	•	→ Pipeline([('preprocessor', …), ('regressor', …)])
	11.	학습 및 전처리 동시 수행

X_train = train.drop('SalePrice', axis=1)
y_train = train['SalePrice']
X_train_processed = pipeline.fit_transform(X_train, y_train)
X_test_processed  = pipeline.transform(test)


	12.	예측 및 제출 파일 생성

preds = pipeline.predict(test)
submission = pd.DataFrame({'Id': raw_test['Id'], 'SalePrice': preds})
submission.to_csv('submission.csv', index=False)

⸻

이 순서대로 머릿속에 체크리스트를 갖고 코드를 작성해 보시면, 차근차근 필요한 부분들이 자동으로 떠오를 것입니다.
중간 중간 “왜 이걸?”, “다른 대안은?”을 스스로 질문해 보시면 더욱 탄탄한 코드 설계 연습이 됩니다.

In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer                # ▶ 수정: SimpleImputer import 확인
from sklearn.ensemble import RandomForestRegressor

# ────────────────────────────────────────────────────────────
# 0) Kaggle 환경인지 로컬 환경인지 구분
if os.path.exists('/kaggle/input/house-prices-advanced-regression-techniques'):
    DATA_DIR = '/kaggle/input/house-prices-advanced-regression-techniques'
else:
    DATA_DIR = os.path.expanduser(
        '/Users/rick/Library/Mobile Documents/com~apple~CloudDocs/'
        'Study/통계&빅데이터/Kaggle/house_price/data/'
    )

# 1) 데이터 로드
raw_train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
raw_test  = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

# ▶ 수정: train/test 복제본 주석 명확히
train = raw_train.copy()    # 학습용 복제본
test  = raw_test.copy()     # 전처리·예측용 복제본

# 2) 데이터 확인
print('▶ train shape:', train.shape)
print('▶ test  shape:',  test.shape)

print("\n▶ [Train Columns] (총 {}개)".format(len(train.columns)))
print(train.columns.tolist())

print("\n▶ [Missing Values in Train] (상위 20개)")
print(train.isnull().sum().sort_values(ascending=False).head(20))

print("\n▶ [Descriptive Statistics (Train)]")
print(train.describe(include='all').T)

# 3) 안전한 전처리를 위한 Pipeline 구성

# 3-1) 수치형·범주형 피처 자동 선택
numeric_features     = train.select_dtypes(include=[np.number]).columns.drop('SalePrice')
categorical_features = train.select_dtypes(include=['object']).columns

# 3-2) 수치형 변환기: 결측치 → 스케일링
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),   # 중앙값으로 결측치 대체
    ('scaler',  StandardScaler()),                   # 평균0·분산1 스케일링
])

# ▶ 수정: 범주형에도 결측치 대체 단계 추가한 Pipeline
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  
    ('onehot',  OneHotEncoder(handle_unknown='ignore')),
])

# 3-3) ColumnTransformer 로 “수치형/범주형 전처리” 결합
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer,     numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ],
    remainder='passthrough'   # 그 외 컬럼(Id 등)은 그대로 통과
)

# 4) 전체 파이프라인: 전처리 → 모델
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor',     RandomForestRegressor(n_estimators=100, random_state=42)),
])

# 5) 학습 데이터 준비
X_train = train.drop(columns='SalePrice')
y_train = train['SalePrice']

# ▶ 수정: fit과 transform을 한 번에, 그리고 테스트 데이터에도 동일 적용
X_train_processed = pipeline.fit_transform(X_train, y_train)  # 전처리+모델 학습
X_test_processed  = pipeline.transform(test)                  # 전처리만

# 6) 예측 및 제출 파일 생성
preds = pipeline.predict(test)
submission = pd.DataFrame({'Id': raw_test['Id'], 'SalePrice': preds})
submission.to_csv('submission_house.csv', index=False)

In [5]:
# 교차 검증
from sklearn.model_selection import cross_val_score, KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X_train, y_train,
                         scoring='neg_root_mean_squared_error',
                         cv=kf)
print("CV RMSE: ", -scores.mean(), "±", scores.std())

CV RMSE:  30363.703394855023 ± 6930.39801626804
